In [1]:
!pip install rapidfuzz

import pandas as pd
import os
from rapidfuzz import fuzz  # 替代 fuzzywuzzy
from datetime import datetime
from google.colab import drive
from tqdm import tqdm  # 進度條，方便查看進度
import numpy as np

# 掛載 Google Drive
drive.mount('/content/drive')

# 設定檔案路徑
base_dir = '/content/drive/My Drive/Datasets/'
big_dataset_paths = [
    base_dir + 'largeds1.csv',
    base_dir + 'largeds2.csv',
    base_dir + 'largeds3.csv',
    base_dir + 'largeds4.csv',
    base_dir + 'largeds5.csv',
    base_dir + 'largeds6.csv'
]
small_dataset_path = base_dir + 'smallds.csv'
output_dir = base_dir

# 檢查檔案是否存在
for path in big_dataset_paths + [small_dataset_path]:
    if os.path.exists(path):
        print(f"檔案存在: {path}")
    else:
        print(f"檔案不存在: {path}")

# 確保輸出資料夾存在
os.makedirs(output_dir, exist_ok=True)

# 生成唯一的輸出檔案名稱（使用時間戳）
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_path = os.path.join(output_dir, f'merged_dataset_{timestamp}.csv')

# 載入小資料集
try:
    small_dataset = pd.read_csv(small_dataset_path, encoding='utf-8', dtype={0: str}, low_memory=False)
except UnicodeDecodeError:
    print("小資料集編碼錯誤，嘗試 utf-8-sig")
    small_dataset = pd.read_csv(small_dataset_path, encoding='utf-8-sig', dtype={0: str}, low_memory=False)

# 初始化合併後的大資料集
big_dataset_combined = pd.DataFrame()

# 載入並合併所有大資料集
for big_path in big_dataset_paths:
    try:
        big_dataset = pd.read_csv(big_path, encoding='utf-8', dtype={0: str}, low_memory=False)
    except UnicodeDecodeError:
        print(f"大資料集 {big_path} 編碼錯誤，嘗試 utf-8-sig")
        big_dataset = pd.read_csv(big_path, encoding='utf-8-sig', dtype={0: str}, low_memory=False)
    big_dataset_combined = pd.concat([big_dataset_combined, big_dataset], ignore_index=True)

# 檢查資料集大小和欄位
print(f"合併後大資料集行數：{len(big_dataset_combined)}")
print(f"小資料集行數：{len(small_dataset)}")
print("大資料集欄位：", big_dataset_combined.columns.tolist())
print("小資料集欄位：", small_dataset.columns.tolist())

# TODO: 根據列印的欄位名稱，更新以下變數
company_name_col = '公司名稱'  # 替換為實際的公司名稱欄位
industry_col = '產業類別'      # 替換為實際的產業類別欄位

# 確保公司名稱為字串格式並去除多餘空格
big_dataset_combined[company_name_col] = big_dataset_combined[company_name_col].astype(str).str.strip()
small_dataset[company_name_col] = small_dataset[company_name_col].astype(str).str.strip()

# 清理名稱：去除常見後綴以提高匹配率
big_dataset_combined[company_name_col] = big_dataset_combined[company_name_col].str.replace(
    r'股份有限公司|有限公司|股份公司|$$股$$|公司', '', regex=True).str.lower()
small_dataset[company_name_col] = small_dataset[company_name_col].str.replace(
    r'股份有限公司|有限公司|股份公司|$$股$$|公司', '', regex=True).str.lower()

# 處理空值或無效名稱
big_dataset_combined = big_dataset_combined[big_dataset_combined[company_name_col].notna() & (big_dataset_combined[company_name_col] != '')]
small_dataset = small_dataset[small_dataset[company_name_col].notna() & (small_dataset[company_name_col] != '')]

# 優化 1：去除大資料集中的重複公司名稱，保留第一筆產業類別
big_dataset_combined = big_dataset_combined.drop_duplicates(subset=[company_name_col], keep='first')
print(f"去重後大資料集行數：{len(big_dataset_combined)}")

# 優化 2：根據公司名稱的首字進行分組，減少比對次數
# 為公司名稱新增首字欄位
big_dataset_combined['first_char'] = big_dataset_combined[company_name_col].str[0]
small_dataset['first_char'] = small_dataset[company_name_col].str[0]

# 模糊比對函數（使用 rapidfuzz 的 partial_ratio 加速）
def find_best_match(name, name_list, min_score=85):
    best_match = None
    best_score = min_score
    best_industry = None
    for _, row in name_list.iterrows():
        score = fuzz.partial_ratio(name, row[company_name_col])  # 使用 rapidfuzz 的 partial_ratio
        if score > best_score:
            best_score = score
            best_match = row[company_name_col]
            best_industry = row[industry_col]
    return best_match, best_score, best_industry

# 分批處理小資料集，並按首字分組比對
batch_size = 1000
results = []
unique_first_chars = small_dataset['first_char'].unique()

for start in range(0, len(small_dataset), batch_size):
    batch = small_dataset.iloc[start:start + batch_size].copy()
    batch['matched_name'] = None
    batch['match_score'] = None
    batch[industry_col] = None

    # 對每個批次中的資料，根據首字過濾大資料集
    for idx, row in tqdm(batch.iterrows(), total=len(batch), desc=f"處理批次 {start//batch_size + 1}"):
        name = row[company_name_col]
        first_char = row['first_char']
        # 只比對相同首字的大資料集子集
        filtered_big_list = big_dataset_combined[big_dataset_combined['first_char'] == first_char]
        if not filtered_big_list.empty:
            best_match, best_score, best_industry = find_best_match(name, filtered_big_list)
        else:
            best_match, best_score, best_industry = None, 0, None
        batch.at[idx, 'matched_name'] = best_match
        batch.at[idx, 'match_score'] = best_score
        batch.at[idx, industry_col] = best_industry
    results.append(batch)

# 合併所有批次結果
merged_dataset = pd.concat(results)

# 移除臨時欄位（僅保留公司名稱和產業類別）
merged_dataset = merged_dataset.drop(columns=['matched_name', 'match_score', 'first_char'])

# 儲存結果，使用 utf-8-sig 編碼
merged_dataset.to_csv(output_path, encoding='utf-8-sig', index=False)
print(f"已完成模糊比對，小資料集已補充產業類別，儲存為 {output_path}")

# 檢查缺失值
min_score = 85  # 與 find_best_match 中定義的一致
missing_count = merged_dataset[industry_col].isna().sum()
if missing_count > 0:
    print(f"警告：有 {missing_count} 筆資料的產業類別缺失，可能是公司名稱在大資料集中無匹配或相似度低於 {min_score}。")
    missing_output_path = os.path.join(output_dir, f'missing_industry_{timestamp}.csv')
    missing_data = merged_dataset[merged_dataset[industry_col].isna()]
    missing_data.to_csv(missing_output_path, encoding='utf-8-sig', index=False)
    print(f"缺失產業類別的資料已儲存為 {missing_output_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.7 MB/s eta 0:00:00
Mounted at /content/drive
檔案存在: /content/drive/My Drive/Datasets/largeds1.csv
檔案存在: /content/drive/My Drive/Datasets/largeds2.csv
檔案存在: /content/drive/My Drive/Datasets/largeds3.csv
檔案存在: /content/drive/My Drive/Datasets/largeds4.csv
檔案存在: /content/drive/My Drive/Datasets/largeds5.csv
檔案存在: /content/drive/My Drive/Datasets/largeds6.csv
檔案存在: /content/drive/My Drive/Datasets/smallds.csv
合併後大資料集行數：4937032
小資料集行數：5831
大資料集欄位： ['統一編號', '公司名稱', '產業類別']
小資料集欄位： ['項目名稱', '選舉名稱', '申報序號／年度', '交易日期', '收支科目', '公司名稱', '身分證／統一編號', '收入金額', '支出金額', '支出用途', '金錢類', '地址', '聯絡電話', '捐贈方式', '存入專戶日期', '返還/繳庫', '應揭露之支出對象', '支出對象之內部人員姓名', '支出對象之內部人員職稱', '政黨之內部人員姓名', '政黨之內部人員職稱', '關係', '更正註記', '資料更正日期']
去重後大資料集行數：907312


處理批次 6: 100%|██████████| 831/831 [02:16<00:00,  6.07it/s]

已完成模糊比對，小資料集已補充產業類別，儲存為 /content/drive/My Drive/Datasets/merged_dataset_20250505_041910.csv
警告：有 1057 筆資料的產業類別缺失，可能是公司名稱在大資料集中無匹配或相似度低於 85。
缺失產業類別的資料已儲存為 /content/drive/My Drive/Datasets/missing_industry_20250505_041910.csv
